### Using JSON Mode and Function Calling for SQL Querying

In [1]:
# %pip install groq
# %pip install sqlparse
# %pip install duckdb


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 7.3 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from groq import Groq
import os 
import json
import sqlparse
from IPython.display import Markdown
import duckdb
import glob
import yaml
print("Modules Imported!")

Modules Imported!


In [13]:
client = Groq(api_key = os.getenv('GROQ_API_KEY'))
model = 'llama-3.3-70b-versatile'

#### Text-To-SQL
The first method is a standard Text-To-SQL implementation. With Text-To-SQL, we describe the database schema to the LLM, ask it to answer a question, and let it write an on-the-fly SQL query against the database to answer that question. Let's see how we can use the Groq API to build a Text-To-SQL pipeline:

First, we have our system prompt. A system prompt is an initial input or instruction given to the model, setting the context or specifying the task it needs to perform, essentially guiding the model's response generation. In our case, our system prompt will serve 3 purposes:

Provide the metadata schemas for our database tables
Indicate any relevant context or tips for querying the DuckDB language or our database schema specifically
Define our desired JSON output (note that to use JSON mode, we must include 'JSON' in the prompt)

In [6]:
system_prompt = '''
You are Groq Advisor, and you are tasked with generating SQL queries for DuckDB based on user questions about data stored in two tables derived from CSV files:

Table: employees.csv
Columns:
employee_id (INTEGER): A unique identifier for each employee.
name (VARCHAR): The full name of the employee.
email (VARCHAR): employee's email address

Table: purchases.csv
Columns:
purchase_id (INTEGER): A unique identifier for each purchase.
purchase_date (DATE): Date of purchase
employee_id (INTEGER): References the employee_id from the employees table, indicating which employee made the purchase.
amount (FLOAT): The monetary value of the purchase.
product_name (STRING): The name of the product purchased

Given a user's question about this data, write a valid DuckDB SQL query that accurately extracts or calculates the requested information from these tables and adheres to SQL best practices for DuckDB, optimizing for readability and performance where applicable.

Here are some tips for writing DuckDB queries:
* DuckDB syntax requires querying from the .csv file itself, i.e. employees.csv and purchases.csv. For example: SELECT * FROM employees.csv as employees
* All tables referenced MUST be aliased
* DuckDB does not implicitly include a GROUP BY clause
* CURRENT_DATE gets today's date
* Aggregated fields like COUNT(*) must be appropriately named

And some rules for querying the dataset:
* Never include employee_id in the output - show employee name instead

Also note that:
* Valid values for product_name include 'Tesla','iPhone' and 'Humane pin'


Question:
--------
{user_question}
--------
Reminder: Generate a DuckDB SQL to answer to the question:
* respond as a valid JSON Document
* [Best] If the question can be answered with the available tables: {"sql": <sql here>}
* If the question cannot be answered with the available tables: {"error": <explanation here>}
* Ensure that the entire output is returned on only one single line
* Keep your query as simple and straightforward as possible; do not use subqueries
'''

Now we will define a text_to_sql function which takes in the system prompt and the user's question and outputs the LLM-generated DuckDB SQL query. Note that since we are using Groq API's JSON mode to format our output, we must indicate our expected JSON output format in either the system or user prompt.

In [7]:
def text_to_sql(client,system_prompt,user_question,model):

    completion = client.chat.completions.create(
        model = model,
        response_format = {"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_question
            }
        ]
    )
  
    return completion.choices[0].message.content

...and a function for executing the DuckDB query that was generated:



In [8]:
def execute_duckdb_query(query):
    original_cwd = os.getcwd()
    os.chdir('data')
    
    try:
        conn = duckdb.connect(database=':memory:', read_only=False)
        query_result = conn.execute(query).fetchdf().reset_index(drop=True)
    finally:
        os.chdir(original_cwd)


    return query_result

Now, we can query our database just by asking a question about the data. Here, the LLM generates a valid SQL query that reasonably answers the question:

In [14]:
user_question = 'What are the most recent purchases?'


llm_response = text_to_sql(client,system_prompt,user_question,model)
sql_json = json.loads(llm_response)
parsed_sql = sqlparse.format(sql_json['sql'], reindent=True, keyword_case='upper')
formatted_sql = f"```sql\n{parsed_sql.strip()}\n```"
display(Markdown(formatted_sql)) 

execute_duckdb_query(parsed_sql)

```sql
SELECT p.purchase_date,
       e.name,
       p.product_name,
       p.amount
FROM purchases.csv AS p
INNER JOIN employees.csv AS e ON p.employee_id = e.employee_id
ORDER BY p.purchase_date DESC
```

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,purchase_date,name,product_name,amount
0,2024-02-05,Jared Dunn,Tesla,75000
1,2024-02-04,Bertram Gilfoyle,iPhone,700
2,2024-02-03,Dinesh Chugtai,Humane pin,500
3,2024-02-02,Erlich Bachman,Tesla,70000
4,2024-02-01,Richard Hendricks,iPhone,750


Note, however, that due to the non-deterministic nature of LLMs, we cannot guarantee a reliable or consistent result every time. I might get a different result than you, and I might get a totally different query tomorrow. How should "most recent purchases" be defined? Which fields should be returned?

Obviously, this is not ideal for making any kind of data-driven decisions. It's hard enough to land on a reliable source-of-truth data model, and even harder when your AI analyst cannot give you a consistent result. While text-to-SQL can be great for generating ad-hoc insights, the non-determinism feature of LLMs makes raw text-to-SQL an impractical solution for a production environment.